In [1]:
import os
import random
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from athena import Athena
import matplotlib.pyplot as plt

# How to Run Simulations with Athena

### First Step: Initialize Simulation Parameters

Athena enables you to simulate Single Cell CRISPR Screens. Before you can run simulations you need to provide Athena with simulation parameters most noteably: on target activity, off target activity, type of CRISPR pertubation, number of Transcription Factors (TFs), number of E-genes, number of kinases, and the number of signalling cascades you want to simulate. Additional parameters you can provide include the number of cells per condition to simulate to where directory to store simulation data.

It's important to note that when initializing the Athena class is when you pass the parameters to Athena. All methods afterwards do not require or do not take any parameters.

In [2]:
athena = Athena(negenes=200,
                nkinases=15,
                ncascades=3, 
                ncpus=3, nbatches=1,
                ngrnas_per_target=1,
                ncells_per_condition=1000)
                # crispr_type='Knockout',


# athena.create_network()
# athena.initialize_kinetics()

Initiate Environmental Parameters...
Check the caches...
Setup Simulator Directory...
Check Network Parameters...
Check gRNA Parameters...
Check Simulation Parameters...
Check Downsampling Parameters...


### Second Step: Create Gene Regulatory Network and Signalling Cascade

Both Gene Regulator Networks and Signalling Cascades are created using Modular Sampling method originally developed by GeneNetWeaver. This modular sampling method allows us to capture the scale-free and modular nature of biologically networks in comparison to an other scale-free random sampling method. Thereby allowing us to better represent the flow between reactions that occurs in biological networks.

**No Parameters Avaliable**

In [3]:
athena.create_network()

Created Network...


### Third Step:  Initialize Reaction Kinectics

Next we need to initialize reaction kinetics of both our GRN and Signalling Cascade.

**No Parameters Avaliable**

In [4]:
athena.initialize_kinetics()

Initialized Kinetics...


### Fourth Step: Create Guide RNA Metadata

Creating gRNA Metadata and Perturbation Coefficient Matrix.

**Parameters Avaliable**

- crispr_type: The type of CRISPR perturbation that you want to simulate. Only three types of CRISPR perturbations are currently supported: Interference, Activation, and Knockout

In [5]:
athena.generate_grnas(crispr_type='Interference', on_target=0.0, off_target=0)
# print (sim_name, gene, gene_on_target, value, ko_combos[col])

100%|██████████| 25/25 [00:00<00:00, 59.58it/s] 


### Fifth Step:  Compile Reactions for Simulation

Once we have initialized the kinetics and create gRNA metadata we need to convert all of this information in complied affinity and propensity probability calculation. This is required for the novel GPU Based Gillespie Algorithm.

**No Parameters Avaliable**

In [6]:
athena.compiling()

Simulation: athena Starting to Processing Batch 0...
Simulation: athena Current Memory % Usage: 3.4 Finished Processing Batch 0...


### Sixith Step: Run Gillispie Tau Leaping Algorithm

Now that we have compiled the chemical reactions we can run our simulations. Athena uses a opencl implementation of the Gillespie Tau-Leaping Algorithm. Which means Athena is CPU and GPU compatible. However, for obvious reasons using a GPU dramatically increases the speed of any simulation you want to run.

**No Parameters Avaliable**

In [ ]:
athena.run()

Batch For Loop:   0%|          | 0/1 [00:00<?, ?it/s]

Creating Command Queue...
Start Simulation...



Running Simulations Batch: 0: 100%|█████████▉| 5980/6000 [05:38<00:00, 26.05it/s]

### Seventh Step: Sample Cells and Downsample Counts

Now that we have simulated cells we can randomly sample our cells and downsample our cells mRNA counts. Athena downsamples simulated cells mRNA counts using the a reference single cell dataset to parameterize both the number of mRNAs the cell will have (also known as a cells library size) and probability that a given mRNA of a given gene is sampled. 

**Parameters Avaliable**

- ncells: number of cells you want to sample from simulated cells
- pop_fp: file path to a loom file that is scanpy compatible that contains single cell dataset. If provided will use the provided dataset to downsample counts instead of internal single cell populations.

In [ ]:
cells_meta, gene_expr = athena.sample(ncells=1000, sim_fp='athena/results/simulated_counts.csv.gz')

### Checking Library Sizes

In [ ]:
cells_meta['mat_lib_size'] = gene_expr.sum(axis=1)
cells_meta[['lib_size', 'mat_lib_size']]

# Visualizing Gene Expression of Gillespie SSA

Now that we conducted a simulation let's see the effects our perturbations have upon our target genes. As we can see when using a Knockout CRISPR type all perturbed cells have count of zero for our target gene. In addition, these perturbations dramatically alter the expression level of multiple genes demonstrating that our simulated cells have been perturbed.

In [ ]:
cell_sim_meta = pd.read_csv(f'{athena.results_dir}/cell_metadata.csv.gz')
simulations = pd.read_csv(f'{athena.results_dir}/simulated_counts.csv.gz')

grna_meta = cell_sim_meta.drop_duplicates(subset=['sim_name']).copy()
grna_meta['perturbed_gene'] = grna_meta['sim_name'].apply(lambda x: x.split('-grna')[0])
genes_to_plot = list("mol_mrna_" + grna_meta['perturbed_gene'].loc[grna_meta.perturbed_gene != 'CTRL'].values)

In [ ]:
df = {"mol_counts": [], "mol": [], "time_step": [], "sim_index": [], "perturbed_gene": []}

for index, row in grna_meta.iterrows():
    
    for gene in genes_to_plot:
        index_locs = cell_sim_meta.loc[cell_sim_meta.sim_i == row.sim_i].index.values
        sim = simulations[gene].iloc[index_locs]
        sim = sim.reset_index(drop=True)
        
        df['time_step'] = df['time_step'] + list(sim.index)
        df['mol_counts'] = df['mol_counts'] + list(sim.values)
        df['mol'] = df['mol'] + [gene] * len(list(sim.values))
        df['sim_index'] = df['sim_index'] + [row.sim_i] * len(list(sim.values))
        df['perturbed_gene'] = df['perturbed_gene'] + [row.perturbed_gene] * len(list(sim.values))
    
df = pd.DataFrame(df)

In [ ]:
for gene in tqdm(df.perturbed_gene.unique()):
    gene_df = df.loc[df.perturbed_gene == gene]
    
    plt.figure(figsize=(9, 6))
    sns.lineplot(data=gene_df, x='time_step', y='mol_counts', hue='mol')
    plt.legend(bbox_to_anchor=(1,1), loc="upper left")
    plt.title(f'Perturbed Gene: {gene}')
    plt.show()

In [ ]:
for gene in tqdm(df.perturbed_gene.unique()):
    
    if gene == athena.ctrl_label:
        continue
    
    mol = 'mol_mrna_' + gene
    gene_df = df.loc[df.perturbed_gene == gene]
    mol_counts = gene_df.loc[gene_df.mol == mol, 'mol_counts'].unique()
    
    print (f"Perturbed Gene: {gene} Unique Counts: {mol_counts}")